In [1]:
# !pip3.10 install -e .

In [2]:
import transformers

transformers

/home/husein/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'transformers' from '/home/husein/mesolitica/t5-static-cache/src/transformers/__init__.py'>

In [3]:
import torch
from transformers.cache_utils import T5StaticCache
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_id      = "mesolitica/translation-t5-small-standard-bahasa-cased-v2"
compute_dtype = torch.bfloat16
device        = "cuda:0"

In [4]:
model     = T5ForConditionalGeneration.from_pretrained(model_id, torch_dtype=compute_dtype) 
tokenizer = T5Tokenizer.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
_ = model.cuda()

In [6]:
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import *
from hqq.utils.patching import prepare_for_inference
import hqq.models.base as hqq_base

hqq_base._QUANT_LAYERS = [torch.nn.Linear, HQQLinear]

quant_config = BaseQuantizeConfig(nbits=4, group_size=64, quant_scale=False, quant_zero=False, axis=1) 
HQQLinear.set_backend(HQQBackend.PYTORCH)

# AutoHQQHFModel.quantize_model(model.model.encoder, quant_config=quant_config, compute_dtype=compute_dtype, device=device)
# AutoHQQHFModel.quantize_model(model.model.decoder, quant_config=quant_config, compute_dtype=compute_dtype, device=device)

# AutoHQQHFModel.set_auto_linear_tags(model.model.encoder)
# prepare_for_inference(model.model.encoder)

# AutoHQQHFModel.set_auto_linear_tags(model.model.decoder)
# prepare_for_inference(model.model.decoder, backend='torchao_int4')

In [7]:
# model.encoder.forward = torch.compile(model.encoder.forward, mode='reduce-overhead', fullgraph=True)

In [8]:
cache = None

In [9]:
%%time
with torch.no_grad():
    s = 'SHAH ALAM Malaysia - Pertubuhan Kebajikan Anak Bersatu Selangor bersetuju pihak kerajaan mewujudkan Suruhanjaya Siasatan Diraja untuk menyiasat isu kartel daging.'
    input_ids = tokenizer(f'terjemah ke Inggeris: {s}', return_tensors = 'pt', max_length = 2048, padding = 'max_length').to('cuda')
    out_encoder = model.encoder(**input_ids)
    out_encoder = out_encoder[0].clone()
    encoder_attention_mask = input_ids['attention_mask']
    labels = torch.tensor([[model.config.decoder_start_token_id]], device = device)
    out_decoder = model.decoder(labels, encoder_hidden_states=out_encoder,
                               past_key_values = None, 
                               encoder_attention_mask = encoder_attention_mask,use_cache = True)
    past_key_values = out_decoder.past_key_values
    proj = model.lm_head(out_decoder.last_hidden_state[:,-1:]).argmax(-1).clone()
    s = tokenizer.decode(proj[0])
    if cache is None:
        cache = T5StaticCache(model.config, compute_dtype, device, past_key_values, 
                  maxlen = input_ids['input_ids'].shape[1])
    else:
        cache.reset(past_key_values)
    seq_length = past_key_values[0][0].shape[2]
    cache_position = seq_length

CPU times: user 337 ms, sys: 134 ms, total: 471 ms
Wall time: 469 ms


In [10]:
def decode_one_tokens(
    model,
    cur_token,
    encoder_attention_mask,
    past_key_values, 
    cache_position, 
    out_encoder,
):
    out_decoder = model.decoder(
        cur_token,
        encoder_hidden_states=out_encoder,
        past_key_values=past_key_values,
        return_dict = False,
        cache_position = cache_position,
        encoder_attention_mask = encoder_attention_mask,
        use_cache = True,
    )
    sequence_output = out_decoder[0]
    sequence_output = sequence_output * (model.model_dim**-0.5)
    lm_logits = model.lm_head(sequence_output)
    new_token = lm_logits.argmax(-1)[:,-1:]
    return new_token

In [11]:
import time

In [15]:
%%time
with torch.no_grad():
    s = 'Pertubuhan Kebajikan Anak Bersatu Selangor bersetuju pihak kerajaan mewujudkan Suruhanjaya Siasatan Diraja untuk menyiasat isu kartel daging.'
    input_ids = tokenizer(f'terjemah ke Inggeris: {s}', return_tensors = 'pt', max_length = 2048, padding = 'max_length').to('cuda')
    out_encoder = model.encoder(**input_ids)
    out_encoder = out_encoder[0].clone()
    encoder_attention_mask = input_ids['attention_mask']
    labels = torch.tensor([[model.config.decoder_start_token_id]], device = device)
    out_decoder = model.decoder(labels, encoder_hidden_states=out_encoder,
                               past_key_values = None, 
                               encoder_attention_mask = encoder_attention_mask,use_cache = True)
    past_key_values = out_decoder.past_key_values
    proj = model.lm_head(out_decoder.last_hidden_state[:,-1:]).argmax(-1).clone()
    s = tokenizer.decode(proj[0])
    cache.reset(past_key_values)
    seq_length = past_key_values[0][0].shape[2]
    cache_position = seq_length

CPU times: user 25.5 ms, sys: 879 µs, total: 26.4 ms
Wall time: 25.2 ms


In [16]:
%%time

before = time.time()
with torch.no_grad():
    for i in range(2048):
        proj = decode_one_tokens(
            model = model,
            cur_token = proj,
            encoder_attention_mask = encoder_attention_mask,
            past_key_values = cache,
            cache_position = cache_position,
            out_encoder = out_encoder,
        ).clone()
        s += tokenizer.decode(proj[0])
        print(i, proj, s)
        cache_position += 1
        
        if bool(proj[0] == model.config.eos_token_id):
            break
            
after = time.time() - before

0 tensor([[32100]], device='cuda:0') The 
1 tensor([[1584]], device='cuda:0') The Selangor
2 tensor([[32100]], device='cuda:0') The Selangor 
3 tensor([[485]], device='cuda:0') The Selangor United
4 tensor([[32100]], device='cuda:0') The Selangor United 
5 tensor([[6127]], device='cuda:0') The Selangor United Children
6 tensor([[12]], device='cuda:0') The Selangor United Children'
7 tensor([[16]], device='cuda:0') The Selangor United Children's
8 tensor([[32100]], device='cuda:0') The Selangor United Children's 
9 tensor([[18034]], device='cuda:0') The Selangor United Children's Welfare
10 tensor([[32100]], device='cuda:0') The Selangor United Children's Welfare 
11 tensor([[7620]], device='cuda:0') The Selangor United Children's Welfare Organization
12 tensor([[32100]], device='cuda:0') The Selangor United Children's Welfare Organization 
13 tensor([[2948]], device='cuda:0') The Selangor United Children's Welfare Organization agreed
14 tensor([[32100]], device='cuda:0') The Selangor U

In [17]:
i / after

259.57796367162115